In [2]:
from pathlib import Path
import pandas as pd

# ================= CONFIGURAÇÃO =================
PASTA_EXCEL = Path(r"C:\projetos\superveniencia_piscofins\data\Output\Resumos\excel")
PASTA_PDF   = Path(r"C:\projetos\superveniencia_piscofins\data\Output\Resumos\pdf")
ARQUIVO_TXT = Path(r"C:\projetos\superveniencia_piscofins\data\Input\Resumos\contratos.txt")
SAIDA_EXCEL = Path(r"C:\projetos\superveniencia_piscofins\data\Output\Resumos\relatorio_faltantes.xlsx")
# ===============================================

def ler_contratos_txt(caminho_txt: Path) -> list[str]:
    with caminho_txt.open("r", encoding="utf-8") as f:
        return [linha.strip() for linha in f if linha.strip()]

def contratos_em_pasta(pasta: Path, extensoes: tuple[str]) -> set[str]:
    contratos = set()
    for arquivo in pasta.iterdir():
        if arquivo.is_file() and arquivo.suffix.lower() in extensoes:
            nome = arquivo.stem
            contratos.add(nome)
    return contratos

def contrato_existe_no_nome(contrato: str, nomes_arquivos: set[str]) -> bool:
    return any(contrato in nome for nome in nomes_arquivos)

def main():
    contratos_txt = ler_contratos_txt(ARQUIVO_TXT)

    arquivos_excel = contratos_em_pasta(PASTA_EXCEL, (".xlsx", ".xls"))
    arquivos_pdf   = contratos_em_pasta(PASTA_PDF, (".pdf",))

    registros = []

    for contrato in contratos_txt:
        existe_excel = contrato_existe_no_nome(contrato, arquivos_excel)
        existe_pdf   = contrato_existe_no_nome(contrato, arquivos_pdf)

        if not existe_excel and not existe_pdf:
            status = "Faltando Excel e PDF"
        elif not existe_excel:
            status = "Faltando Excel"
        elif not existe_pdf:
            status = "Faltando PDF"
        else:
            status = "OK"

        registros.append({
            "Contrato": contrato,
            "Existe na pasta Excel": "Sim" if existe_excel else "Não",
            "Existe na pasta PDF": "Sim" if existe_pdf else "Não",
            "Status": status
        })

    df = pd.DataFrame(registros)

    # Mantém apenas os que têm algum problema
    df_faltantes = df[df["Status"] != "OK"]

    df_faltantes.to_excel(SAIDA_EXCEL, index=False)

    print(f"Relatório gerado em: {SAIDA_EXCEL}")

if __name__ == "__main__":
    main()


Relatório gerado em: C:\projetos\superveniencia_piscofins\data\Output\Resumos\relatorio_faltantes.xlsx


0 A ESQUERDA

In [1]:
from pathlib import Path
import pandas as pd
import re
from collections import defaultdict

# ================= CONFIGURAÇÃO =================
PASTA_EXCEL = Path(r"C:\projetos\superveniencia_piscofins\data\Output\Resumos\excel")
ARQUIVO_TXT = Path(r"C:\projetos\superveniencia_piscofins\data\Input\Resumos\contratos.txt")
SAIDA_EXCEL = Path(r"C:\projetos\superveniencia_piscofins\data\Output\Resumos\relatorio_faltantes_excel.xlsx")
# ===============================================

RE_NUM = re.compile(r"\d+")

def normalizar_7dig(s: str) -> str | None:
    dig = re.sub(r"\D", "", str(s).strip())
    if not dig:
        return None
    return dig.zfill(7)

def ler_contratos_txt(caminho_txt: Path) -> list[str]:
    contratos = []
    with caminho_txt.open("r", encoding="utf-8", errors="replace") as f:
        for linha in f:
            linha = linha.strip()
            if not linha:
                continue
            c = normalizar_7dig(linha)
            if c:
                contratos.append(c)

    # remove duplicados preservando ordem
    vistos = set()
    out = []
    for c in contratos:
        if c not in vistos:
            vistos.add(c)
            out.append(c)
    return out

def mapear_contratos_na_pasta_excel(pasta: Path, extensoes: tuple[str, ...]) -> dict[str, set[str]]:
    """
    Retorna: contrato (7 dígitos) -> {arquivos Excel que contêm esse contrato no nome}
    """
    mapa = defaultdict(set)
    for arquivo in pasta.iterdir():
        if not arquivo.is_file():
            continue
        if arquivo.suffix.lower() not in extensoes:
            continue

        for n in RE_NUM.findall(arquivo.stem):
            c = normalizar_7dig(n)
            if c:
                mapa[c].add(arquivo.name)

    return mapa

def main():
    contratos_txt = ler_contratos_txt(ARQUIVO_TXT)
    mapa_excel = mapear_contratos_na_pasta_excel(PASTA_EXCEL, (".xlsx", ".xls"))

    registros = []
    for contrato in contratos_txt:
        existe_excel = contrato in mapa_excel

        status = "OK" if existe_excel else "Faltando Excel"

        registros.append({
            "Contrato": contrato,
            "Existe na pasta Excel": "Sim" if existe_excel else "Não",
            "Status": status,
            "Arquivos Excel (exemplos)": " | ".join(sorted(list(mapa_excel.get(contrato, [])))[:5]),
        })

    df = pd.DataFrame(registros)

    # Mantém apenas os que têm algum problema
    df_faltantes = df[df["Status"] != "OK"].copy()

    SAIDA_EXCEL.parent.mkdir(parents=True, exist_ok=True)
    df_faltantes.to_excel(SAIDA_EXCEL, index=False)

    print(f"Relatório gerado em: {SAIDA_EXCEL}")
    print(f"Total contratos TXT: {len(contratos_txt)}")
    print(f"Total faltando Excel: {len(df_faltantes)}")

if __name__ == "__main__":
    main()


Relatório gerado em: C:\projetos\superveniencia_piscofins\data\Output\Resumos\relatorio_faltantes_excel.xlsx
Total contratos TXT: 22881
Total faltando Excel: 1
